# Python Course Project

*Author: Tanja Platonova*

*Date: 19.9.2024*

# Analyzing Mobile Game Data Using Pandas

### Project Goals

The main goals of my project are to enhance my data analytics portfolio and demonstrate the skills I have gained throughout the course. I selected this project from a collection of test assignments for data analytics.

### Data Analysis Task

The task is to analyze mobile game data, which includes information about players such as their level, country, platform, registration date, and last login date. This analysis is expected to provide insight into the game's audience and player behavior.

### Questions

1) For each platform, determine the top 5 countries by the number of players.
2) What is the maximum lifetime of players in the game?
3) How many players are above level 20?
4) Calculate the average player level for each country.

### Dataset Description

Dataset: [Google Sheets Link](https://docs.google.com/spreadsheets/d/1bb0mBd9OGSdH08POkWgcYlK9a2YPQq1Dd2XyikUdeeI/edit#gid=1478114822) 

The initial table contains the following columns:

* country - the player's country
* pl - the player's level
* plf - the platform
* reg - the player's registration date
* ts - the time of the player's last login to the game
* user - the player's ID

## Data Preparation

At this step I am going to detect and correct (or remove) inaccurate records from a dataset.

In [1]:
# Importing libraries 
import pandas as pd

In [2]:
# Loading the dataset into a dataframe
df = pd.read_csv("cp_data.csv")

In [3]:
# Getting a summary, including column names, non-null counts, and data types
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 6 columns):
 #   Column   Non-Null Count  Dtype 
---  ------   --------------  ----- 
 0   country  1000 non-null   object
 1   pl       1000 non-null   int64 
 2   plf      1000 non-null   object
 3   reg      1000 non-null   int64 
 4   ts       1000 non-null   int64 
 5   user     1000 non-null   object
dtypes: int64(3), object(3)
memory usage: 47.0+ KB


In [4]:
#### Getting a quick look at the data
df.head()

,country,pl,plf,reg,ts,user
0,IT,33,Android,1511191920859,1586044805457,1122239prod-eu
1,IN,33,Android,1509074590554,1586045807421,147065prod-us
2,TR,33,iOS,1575420775882,1586047059117,25564086prod-eu
3,HK,12,Android,1584406076183,1586045087568,10123050prod-as
4,CN,21,Android,1581887511291,1586046421744,9893956prod-as


### Dataset Issues

Upon reviewing the dataset, I identified several issues:

* Unclear column names
* Redundant user IDs
* Non-human-readable date format
* Non-human-readable country codes

In the following sections, I address these issues to improve data quality.

#### Renaming and Reorganizing Columns

To make the dataset more readable, I renamed the columns for better clarity:

In [5]:
# Original column names
df.columns

Index(['country', 'pl', 'plf', 'reg', 'ts', 'user'], dtype='object')

In [6]:
# Renaming and reorganizing the columns
df.columns = ['country_code', 'level', 'platform', 'reg_date', 'last_login', 'user_id']

In [7]:
# Displaying the updated dataframe
df.head()

,country_code,level,platform,reg_date,last_login,user_id
0,IT,33,Android,1511191920859,1586044805457,1122239prod-eu
1,IN,33,Android,1509074590554,1586045807421,147065prod-us
2,TR,33,iOS,1575420775882,1586047059117,25564086prod-eu
3,HK,12,Android,1584406076183,1586045087568,10123050prod-as
4,CN,21,Android,1581887511291,1586046421744,9893956prod-as


The updated column names are:

* country_code: The player's country code
* level: The player's level
* platform: The platform used (e.g., iOS, Android)
* reg_date: The player's registration date
* last_login: The date of the player's last login
* user_id: The player's unique identifier

#### Date Formatting

The date columns contain Unix timestamps in milliseconds, so I converted them to a human-readable format. Since the analysis only requires the date (not the time), I removed the time part from the converted dates.

In [8]:
# Importing necessary module for date formatting
from datetime import datetime

In [9]:
# Columns containing Unix timestamps to be converted
columns_to_convert = ['reg_date', 'last_login']

In [10]:
# Loop through each column and convert the Unix timestamp to a human-readable date format
for column in columns_to_convert:
    df[column] = pd.to_datetime(df[column], unit='ms')
    df[column] = df[column].dt.strftime('%Y-%m-%d')

In [11]:
# Displaying the updated dataframe to verify the changes
df.head()

,country_code,level,platform,reg_date,last_login,user_id
0,IT,33,Android,2017-11-20,2020-04-05,1122239prod-eu
1,IN,33,Android,2017-10-27,2020-04-05,147065prod-us
2,TR,33,iOS,2019-12-04,2020-04-05,25564086prod-eu
3,HK,12,Android,2020-03-17,2020-04-05,10123050prod-as
4,CN,21,Android,2020-02-16,2020-04-05,9893956prod-as


#### Clearing the user_id Column 

Looking more closely, the user_id column appears to contain a region code (eg. "prod-eu", "prod-us") in the last 7 characters. Since this information is not necessary for my analysis, I will extract these values and check them. If they are consistent and irrelevant, I will drop them and keep only the essential part of the user_id.

In [12]:
# Extracting everything except the last 7 characters (numeric part of user_id)
df['user_id_temp'] = df['user_id'].str[:-7]

# Extract the last 7 characters to inspect the region codes
df['temp'] = df['user_id'].str[-7:]

In [13]:
# Checking unique values in the 'temp' column to understand the region codes
print(df["temp"].unique())

['prod-eu' 'prod-us' 'prod-as' 'prod-in']


Since the region codes are not useful for this analysis, I can safely drop the 'temp' column and keep the cleaned 'user_id'

In [14]:
# Droping 'temp' column
our_cols = ['country_code', 'level', 'platform', 'reg_date', 'last_login', 'user_id_temp']

In [15]:
df = df[our_cols]

In [16]:
# Renaming 'user_id_temp' back to 'user_id' to restore the original column name after cleaning
df.columns = ['country_code', 'level', 'platform', 'reg_date', 'last_login', 'user_id']

#### Countries' Full Names

To make the country information more user-friendly, I mapped the country codes in the dataset to their corresponding full country names using ISO-3166 codes. The data for country codes and names was sourced from the [ISO-3166 dataset](https://github.com/lukes/ISO-3166-Countries-with-Regional-Codes/tree/master/all).

In [17]:
# Loading countries.csv, containing country codes and names
countries = pd.read_csv("cp_countries.csv")

# Displaying the contents of the file to ensure it loaded correctly
print(countries)

          country_name country_code
0          Afghanistan           AF
1        Åland Islands           AX
2              Albania           AL
3              Algeria           DZ
4       American Samoa           AS
..                 ...          ...
244  Wallis and Futuna           WF
245     Western Sahara           EH
246              Yemen           YE
247             Zambia           ZM
248           Zimbabwe           ZW

[249 rows x 2 columns]


I merged the main dataset with the countries.csv file using a left join to ensure that all records in the main dataset were preserved, even if the country code was missing in the countries.csv.

In [18]:
# Merging the main dataset with the country names
df_merged = df.merge(countries[['country_name','country_code']], on = 'country_code', how = 'left')
# Checking the first few rows of the merged dataset
df_merged.head()

,country_code,level,platform,reg_date,last_login,user_id,country_name
0,IT,33,Android,2017-11-20,2020-04-05,1122239,Italy
1,IN,33,Android,2017-10-27,2020-04-05,147065,India
2,TR,33,iOS,2019-12-04,2020-04-05,25564086,Türkiye
3,HK,12,Android,2020-03-17,2020-04-05,10123050,Hong Kong
4,CN,21,Android,2020-02-16,2020-04-05,9893956,China


In [19]:
# Getting an overview of the merged dataframe, including the new 'country_name' column
df_merged.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1000 entries, 0 to 999
Data columns (total 7 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   country_code  1000 non-null   object
 1   level         1000 non-null   int64 
 2   platform      1000 non-null   object
 3   reg_date      1000 non-null   object
 4   last_login    1000 non-null   object
 5   user_id       1000 non-null   object
 6   country_name  997 non-null    object
dtypes: int64(1), object(6)
memory usage: 54.8+ KB


Upon inspecting the merged dataframe, I found three rows where the country_name was missing (NaN). This was caused by invalid country codes in the original dataset. Since the number of affected rows is small and insignificant for the analysis, I decided to remove them.

In [20]:
# Checking which rows have missing country names
df_merged[df_merged['country_name'].isna()]

,country_code,level,platform,reg_date,last_login,user_id,country_name
332,UNKNOWN,4,Android,2020-03-30,2020-04-05,16307174,NaN
662,XK,24,Android,2020-02-06,2020-04-05,27571378,NaN
696,UNKNOWN,8,iOS,2020-03-10,2020-04-05,15959514,NaN


In [21]:
# Dropping rows with missing 'country_name' values
df_final = df_merged.dropna()
# Verifying the final cleaned DataFrame
df_final.info()

<class 'pandas.core.frame.DataFrame'>
Index: 997 entries, 0 to 999
Data columns (total 7 columns):
 #   Column        Non-Null Count  Dtype 
---  ------        --------------  ----- 
 0   country_code  997 non-null    object
 1   level         997 non-null    int64 
 2   platform      997 non-null    object
 3   reg_date      997 non-null    object
 4   last_login    997 non-null    object
 5   user_id       997 non-null    object
 6   country_name  997 non-null    object
dtypes: int64(1), object(6)
memory usage: 62.3+ KB


In [22]:
df_final.head(5)

,country_code,level,platform,reg_date,last_login,user_id,country_name
0,IT,33,Android,2017-11-20,2020-04-05,1122239,Italy
1,IN,33,Android,2017-10-27,2020-04-05,147065,India
2,TR,33,iOS,2019-12-04,2020-04-05,25564086,Türkiye
3,HK,12,Android,2020-03-17,2020-04-05,10123050,Hong Kong
4,CN,21,Android,2020-02-16,2020-04-05,9893956,China


## Exploring and Analysing the Data

Now that my dataset is clean and prepared, I can begin to answer the questions I listed earlier. But first, let's look at some basic statistics to better understand the data.

In [23]:
# Displaying basic statistics for numeric columns
df_final.describe()

,level
count,997.000000
mean,13.083250
std,10.094737
min,1.000000
25%,4.000000
50%,10.000000
75%,20.000000
max,33.000000


In [24]:
# Checking the number of unique users
unique_users = df_final['user_id'].nunique()
print("Number of unique users:", unique_users)

Number of unique users: 997


In [25]:
# Checking numbers of unique countries
print("Number of unique countries:", df_final['country_code'].nunique())

Number of unique countries: 100


The dataset contains 997 unique players with an average level of 13, from 100 different countries. With this initial exploration, I now have a better understanding of the scope of the data, and I can proceed with the more detailed analysis.

### Top 5 Countries by Number of Players

To identify the top 5 countries with the most players for each platform, I grouped the dataset by platform and country, and counted the number of players for each combination. Then I sorted the results by platform and player count, ensuring the counts are in descending order within each platform. Finally, I extracted the top 5 countries by player count for each platform.

The result shows the top countries that contribute the most players on each platform.

In [26]:
# Groupping the dataset by platform and country and counting the number of players
country_counts = df_final.groupby(['platform', 'country_name']).size().reset_index(name='player_count')

# Sorting the results by platform and player count in descending order
country_counts_sorted = country_counts.sort_values(by=['platform', 'player_count'], ascending=[True, False])

# Extracting top 5
top_countries_per_platform = country_counts_sorted.groupby('platform').head(5).reset_index(drop=True)

In [27]:
top_countries_per_platform

,platform,country_name,player_count
0,Amazon,Côte d'Ivoire,1
1,Amazon,United States of America,1
2,Android,India,151
3,Android,Russian Federation,59
4,Android,China,58
5,Android,Indonesia,36
6,Android,Philippines,30
7,Samsung,Egypt,3
8,Samsung,Brazil,1
9,Samsung,Costa Rica,1


### Maximum Lifetime of Players in the Game

To calculate the maximum lifetime of players in the game, I will subtract the registration date (reg_date) from the last login date (last_login). This will give me the lifetime (in days) of each player, representing how long they've been active in the game. I will then find the player with the longest lifetime.

In [28]:
# Converting 'last_login' and 'reg_date' columns to datetime format
df_final['last_login'] = pd.to_datetime(df_final['last_login'])
df_final['reg_date'] = pd.to_datetime(df_final['reg_date'])

# Calculating player lifetime as the difference between 'last_login' and 'reg_date' in days
df_final['lifetime'] = (df_final['last_login']-df_final['reg_date']).dt.days

/var/folders/xp/3_ws7nb57575kz8ltkrf3lk00000gn/T/ipykernel_8400/3405633170.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_final['last_login'] = pd.to_datetime(df_final['last_login'])
/var/folders/xp/3_ws7nb57575kz8ltkrf3lk00000gn/T/ipykernel_8400/3405633170.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df_final['reg_date'] = pd.to_datetime(df_final['reg_date'])
/var/folders/xp/3_ws7nb57575kz8ltkrf3lk00000gn/T/ipykernel_8400/3405633170.py:6: SettingWithCopyWarning: 
A value is trying to be set 

In [29]:
# Previewing the first 5 rows of the 'lifetime' column
df_final.head(5)[['lifetime']]

,lifetime
0,867
1,891
2,123
3,19
4,49


In [30]:
# Converting the 'lifetime' to integers for clarity
df_final.loc[:,'lifetime'] = df_final['lifetime'].astype('int')

# Calculating the maximum lifetime
max_lifetime = df_final['lifetime'].max()

In [31]:
print("The answer is:", max_lifetime, "days")

The answer is: 891 days


### How Many Players Are Above Level 20?

To determine how many players are above level 20, I filter the dataset for those whose level is above 20 and then count the unique players.

In [32]:
# Filtering the dataset to count players with a level greater than 20
players_20 = df_final[df_final['level'] > 20]['user_id'].nunique()
print("Number of players above level 20:",players_20)

Number of players above level 20: 249


Out of 997 unique players, only 249 (25%) have successfully reached a level higher than 20. Although this shows an interesting pattern in player progression, further analysis would be required to fully understand the factors influencing player advancement — something that is beyond the scope of this project.

### Average Player Level for Each Country

To calculate the average player level for each country, I grouped the dataset by country_name and calculated the average level for each country. I then sorted the countries in descending order by their average player level.

In [33]:
# Grouping by 'country_name' and calculating the average player level
country_avglevel = df_final.groupby(['country_name'])['level'].mean().reset_index(name='avg_level')

# Sorting in descending order
country_avglevel_sorted = country_avglevel.sort_values(by=['avg_level'], ascending=[False])
country_avglevel_sorted.reset_index(drop=True)

,country_name,avg_level
0,Zambia,33.0
1,Qatar,33.0
2,Austria,33.0
3,Yemen,30.0
4,Mozambique,29.5
...,...,...
95,"Palestine, State of",3.0
96,"Netherlands, Kingdom of the",3.0
97,Jordan,3.0
98,Bulgaria,3.0


In [34]:
# Displaying the top 10 countries by average player level
top_countries_players_level = country_avglevel_sorted.head(10)
top_countries_players_level

,country_name,avg_level
99,Zambia,33.0
72,Qatar,33.0
6,Austria,33.0
98,Yemen,30.0
57,Mozambique,29.5
53,Mauritania,28.0
30,Greece,28.0
25,Ethiopia,28.0
23,Ecuador,24.0
81,Spain,23.0


This gives us the top 10 countries with the highest average player level. The result could be interesting for understanding which countries have more advanced players. However, further analysis may be needed to determine the underlying factors behind these trends, but for now we will focus on displaying the top 10 countries.

## Conclusion

In this project, I explored and analyzed mobile game data using Pandas. By cleaning and organizing the dataset, I was able to answer key questions about player distribution and behavior. The analysis offered valuable insights and provided a solid foundation for understanding the game's player base and engagement trends.

Throughout the analysis, I applied several skills I learned in this course, including working with data types, manipulating strings, using loops, and working with lists. This project not only expanded my knowledge, but also highlighted the practical application of these concepts in real-world data analysis.